# Setup for doing inference

# Client Side

In [1]:
# # **************** UNCOMMENT AND RUN THIS CELL IF YOU NEED TO INSTALL/ UPGRADE TSAI ****************
#stable = True # Set to True for latest pip version or False for main branch in GitHub
#!pip install {"tsai -U" if stable else "git+https://github.com/timeseriesAI/tsai.git"} >> /dev/null

In [5]:
import pandas as pd

In [6]:
new_df = pd.read_csv('cex4WindDataInterpolated.csv')
new_df['t'] = pd.to_datetime(new_df['t'])

fcst_date = "2003-01-01"
fcst_history = 104
freq = '24H'
datetime_col = "t"

dates = pd.date_range(start=None, end=fcst_date, periods=fcst_history, freq=freq)
dates

DatetimeIndex(['2002-09-20', '2002-09-21', '2002-09-22', '2002-09-23',
               '2002-09-24', '2002-09-25', '2002-09-26', '2002-09-27',
               '2002-09-28', '2002-09-29',
               ...
               '2002-12-23', '2002-12-24', '2002-12-25', '2002-12-26',
               '2002-12-27', '2002-12-28', '2002-12-29', '2002-12-30',
               '2002-12-31', '2003-01-01'],
              dtype='datetime64[ns]', length=104, freq='24H')

In [7]:
new_df = new_df[new_df[datetime_col].isin(dates)].reset_index(drop=True)
new_df

,t,toy,p,Ws1,Wd1,T1,Ws2,Wd2,T2,Ws3,Wd3,T3
0,2002-09-20,262.0,1.075,9.0,283.4,286.000000,9.0,283.4,286.000,9.0,283.4,286.000000
1,2002-09-21,263.0,NaN,3.8,82.5,285.200000,3.8,82.5,285.200,3.8,82.5,285.200000
2,2002-09-22,264.0,NaN,4.3,96.6,283.300000,4.3,96.6,283.300,4.3,96.6,283.300000
3,2002-09-23,265.0,1.933,8.8,51.9,283.142857,8.8,51.2,283.475,8.6,50.9,283.533333
4,2002-09-24,266.0,4.770,7.5,270.8,284.600000,7.5,270.8,284.600,7.5,270.8,284.600000
...,...,...,...,...,...,...,...,...,...,...,...,...
99,2002-12-28,361.0,7.365,11.7,121.4,275.400000,11.7,121.4,275.400,11.7,121.4,275.400000
100,2002-12-29,362.0,NaN,5.4,27.5,273.300000,5.4,27.5,273.300,5.4,27.5,273.300000
101,2002-12-30,363.0,NaN,10.8,57.0,269.700000,10.8,57.0,269.700,10.8,57.0,269.700000
102,2002-12-31,364.0,NaN,9.7,74.4,265.700000,9.7,74.4,265.700,9.7,74.4,265.700000


# Serverless Function

In [2]:
import sklearn
from tsai.basics import *
my_setup(sklearn)

os              : Linux-5.15.49-linuxkit-aarch64-with-glibc2.36
python          : 3.11.5
tsai            : 0.3.8
fastai          : 2.7.13
fastcore        : 1.5.29
sklearn         : 1.3.2
torch           : 2.1.2
device          : cpu
cpu cores       : 5
threads per cpu : 1
RAM             : 7.76 GB
GPU memory      : N/A


In [8]:
# Check for same versions! (training and inference)
from tsai.inference import load_learner

preproc_pipe = load_object('data/preproc_pipe.pkl')
exp_pipe = load_object('data/exp_pipe.pkl')

learn = load_learner("models/patchTST.pt")
filtered_df = learn.transform(new_df)
filtered_df

Initial memory usage: 9.88 KB   
Final memory usage  : 5.41 KB    (-45.2%)


,t,toy,p,Ws1,Wd1,T1,Ws2,Wd2,T2,Ws3,Wd3,T3
0,2002-09-20,0.940348,-0.626848,0.256933,0.962407,1.071027,0.256976,0.962111,1.070988,0.257288,0.965442,1.070879
1,2002-09-21,0.949947,-0.626848,-1.026068,-1.280310,0.919713,-1.026170,-1.280316,0.919711,-1.025967,-1.274778,0.919606
2,2002-09-22,0.959545,-0.626848,-0.902702,-1.122907,0.560334,-0.902790,-1.122933,0.560418,-0.902577,-1.117550,0.560322
3,2002-09-23,0.969144,-0.468315,0.207587,-1.621908,0.530613,0.207624,-1.629683,0.593514,0.158576,-1.627148,0.604445
4,2002-09-24,0.978742,0.055878,-0.113164,0.821749,0.806225,-0.113162,0.821471,0.806251,-0.112882,0.824940,0.806148
...,...,...,...,...,...,...,...,...,...,...,...,...
99,2002-12-28,1.890608,0.535356,0.923106,-0.846056,-0.933907,0.923225,-0.846118,-0.933463,0.923593,-0.841007,-0.933519
100,2002-12-29,1.900207,0.535356,-0.631298,-1.894294,-1.331112,-0.631356,-1.894221,-1.330573,-0.631119,-1.888079,-1.330617
101,2002-12-30,1.909806,0.535356,0.701048,-1.564975,-2.012028,0.701142,-1.564944,-2.011325,0.701492,-1.559127,-2.011351
102,2002-12-31,1.919404,0.535356,0.429644,-1.370733,-2.768606,0.429707,-1.370727,-2.767721,0.430034,-1.365101,-2.767727


In [17]:
from tsai.basics import *
x_vars = new_df.columns[1:]
y_vars = new_df.columns[1:]
new_X, _ = prepare_forecasting_data(filtered_df, fcst_history=104, fcst_horizon=0, x_vars=x_vars, y_vars=None)
new_X.shape

(1, 11, 104)

In [18]:
new_scaled_preds, *_ = learn.get_X_preds(new_X)

In [19]:
fcst_horizon = 60

new_scaled_preds = to_np(new_scaled_preds).swapaxes(1,2).reshape(-1, len(y_vars))
dates = pd.date_range(start=fcst_date, periods=fcst_horizon + 1, freq='7D')[1:]
preds_df = pd.DataFrame(dates, columns=[datetime_col])
preds_df.loc[:, y_vars] = new_scaled_preds
preds_df = learn.inverse_transform(preds_df)
preds_df

,t,toy,p,Ws1,Wd1,T1,Ws2,Wd2,T2,Ws3,Wd3,T3
0,2003-01-08,328.023254,3.948730,8.272989,111.260529,271.557739,8.272989,111.241608,271.564514,8.268640,111.233467,271.565704
1,2003-01-15,328.682587,4.807099,8.779745,95.376030,271.935638,8.779745,95.374283,271.936523,8.779709,95.373466,271.936676
2,2003-01-22,323.508545,5.353201,9.195480,111.002213,272.477173,9.195480,110.995934,272.481079,9.193743,110.993210,272.481750
3,2003-01-29,328.253479,4.580859,8.580874,126.019501,272.589966,8.580875,126.008232,272.597351,8.578636,126.003387,272.598663
4,2003-02-05,330.052765,4.639828,8.445001,123.780159,273.345215,8.445001,123.767601,273.352234,8.442806,123.762207,273.353455
5,2003-02-12,339.481293,5.039964,9.344320,128.870087,272.127075,9.344320,128.870789,272.125366,9.345852,128.871048,272.125061
6,2003-02-19,337.864990,4.086117,8.579801,117.066032,271.551971,8.579802,117.064873,271.551941,8.580777,117.064339,271.551941
7,2003-02-26,343.374939,4.030464,8.760872,106.116394,271.282593,8.760872,106.104332,271.288483,8.758610,106.099075,271.289520
8,2003-03-05,341.039307,4.968817,9.288982,124.828400,272.172668,9.288982,124.820946,272.174286,9.288424,124.817749,272.174561
9,2003-03-12,349.418457,4.163606,9.116268,110.345856,270.580872,9.116268,110.337593,270.585999,9.114528,110.334038,270.586914


# FastAPI